## Sentiment Analysis using pretrained word model


**IMPORTANT**<br>
Enable **GPU acceleration** by going to *Runtime > Change Runtime Type*. Keep in mind that, on certain tiers, you're not guaranteed GPU access depending on usage history and current load.
<br><br>

To demonstrate word vectors, we are going to use **Gensim**.

In [ ]:
# Upgrade gensim just in case.
!pip install -U gensim==4.*

In [ ]:
import collections
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
import tensorflow as tf

from gensim.models.keyedvectors import KeyedVectors
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

**NOTE**<br>
Gensim library:<br>
https://radimrehurek.com/gensim/models/word2vec.html<br>
https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html



## Using Pretrained, Third-Party Vectors

There are a variety of pretrained, static word vector packages. We will use the **Google News** vectors, a collection of three million, 300-dimension word vectors trained from three billion words from a Google News corpus (circa 2015).

We'll need to first download the actual word vectors. It's over a gigabyte in size but will fit within our environment.

In [ ]:
!gdown "https://drive.google.com/uc?id=1BpfbHu4denceXiv8yfdY3EHgjKIcULku"

In [ ]:
embedding_file = './GoogleNews-vectors-negative300.bin.gz'
embedding_file

Next, we'll have **gensim** load the vectors through the **KeyedVectors** module which will enable us to look up vectors by tokens and indices.<br>
https://radimrehurek.com/gensim/models/keyedvectors.html

We'll train a **Keras** model to use these Google News vectors to perform sentiment analysis on a bunch of **Yelp** reviews.
<br><br>
For the model, we will use 1,000,000 word vectors.



In [ ]:
%%time
word_vectors = KeyedVectors.load_word2vec_format(embedding_file, binary=True, limit=1000000)
word_vectors

The dataset we'll use is *Yelp Polarity Reviews*, a collection of ~600,000 reviews for both training and testing.<br><br>
The original Yelp reviews use a five-star rating system. The ratings in this dataset have been modified to simply be negative (label==1) or positive (label==2).<br>
https://www.tensorflow.org/datasets/catalog/yelp_polarity_reviews<br><br>
Tensorflow comes with a datasets loader but we're going to download the file manually and process the data ourselves for completeness.

In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz"

Unzipping the archive results in *train.csv* and *test.csv* files placed in the default *contents* folder of our environment.

In [ ]:
!tar xvzf /root/input/yelp_review_polarity_csv.tgz

# Show current working directory.
!pwd

The **Pandas** library makes it simple to load a CSV file into memory and manipulate the data.<br>
https://pandas.pydata.org/<br>
https://pandas.pydata.org/docs/<br>
https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html?highlight=read_csv

Here, we're loading the CSV into a Pandas **dataframe** (sort of like an in-memory table) and explicitly naming the columns.<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html?highlight=dataframe#pandas.DataFrame

In [ ]:
yelp_train = pd.read_csv('yelp_review_polarity_csv/train.csv', names=['sentiment', 'review'])
print(yelp_train.shape)

We can get a quick view of the data through the *head* method.<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html

In [ ]:
yelp_train.head()

To save on training time, we'll train on 100,000 reviews rather than the full set. To do that, we'll shuffle the dataset using the *sample* method and *copy* the first 100,000 entries. The reason to shuffle first is to ensure we get a mix of reviews from a variety of businesses (in case the data is sorted in some way).<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html


In [ ]:
TRAIN_SIZE = 100000
yelp_train = yelp_train.sample(frac=1, random_state=1)[:TRAIN_SIZE].copy()
print(yelp_train.shape)

The next thing to do is adjust the labels. This is a **binary classification problem**, so our model's output layer will be a single unit with a **sigmoid** activation function. This function's output will be between 0 and 1 which is then compared against the training label. But the labels are currently 1 for negative, and 2 for positive, which is going to cause problems when calculating the loss.<br><br>
So we'll simply replace the 1s with 0s, and 2s with 1s using the *replace* method.<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html
<br><br>
Alternatively, we could keep the labels as-is and treat this as a **multiclassification** problem with two labels and use a **softmax**, but we would then need to **one-hot encode** the labels.


In [ ]:
yelp_train['sentiment'].replace(to_replace=1, value=0, inplace=True)
yelp_train['sentiment'].replace(to_replace=2, value=1, inplace=True)

In [ ]:
yelp_train.head()

Next, we'll create train/validation splits.<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
yelp_train_split, yelp_val_split = train_test_split(yelp_train, train_size=0.85, random_state=1)
print(yelp_train_split.shape)
print(yelp_val_split.shape)

In [ ]:
# Set up training data.
train_reviews = yelp_train_split['review']
y_train = np.array(yelp_train_split['sentiment'])
print(y_train.shape)
print(train_reviews.shape)

In [ ]:
y_train

In [ ]:
train_reviews[:2]

In [ ]:
# Set up validation data.
val_reviews = yelp_val_split['review']
y_val = np.array(yelp_val_split['sentiment'])
print(y_val.shape)
print(val_reviews.shape)

In [ ]:
val_reviews[:2]

A quick sanity check to see how our data is distributed (e.g. balanced or skewed).

In [ ]:
collections.Counter(y_train)

Because we're relying more on richer encodings (in this case, word vectors), we won't perform as much preprocessing this time around. We'll stick with using the regular Keras **tokenizer** and just filter out numbers and certain symbols.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer<br><br>
We'll also have the tokenizer limit itself to tokenizing only the most frequent 20,000 words. This way, the model will focus on the most frequent descriptive sentiment words.

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=20000,
                                               filters='0123456789!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                                               lower=True)

In [ ]:
# Build the vocabulary.
tokenizer.fit_on_texts(train_reviews)

The next step is to vectorize our reviews.

We are going to use the *text_to_sequences* method to turn each review into a sequence of integers, with each integer representing its corresponding token.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences



In [ ]:
%%time
X_train = tokenizer.texts_to_sequences(train_reviews)

In [ ]:
# The first review in the training set, vectorized.
print(X_train[0])

We can look up the corresponding tokens using the tokenizer's *index_word* dict. Here are the tokens corresponding to the first three integers from the first vectorized review.

In [ ]:
[tokenizer.index_word[x] for x in X_train[0][:3]]

We can also convert the integer sequence back to text using the *sequences_to_texts* method, and compare it against the original text.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#sequences_to_texts

In [ ]:
# Review excerpt reconstructed from integer sequence.
tokenizer.sequences_to_texts([X_train[0]])[0][:300]

In [ ]:
# Original review text.
train_reviews.iloc[0][:300]

Some models and situations require us to **pad** our sequences to the same length. While that's not the case here, it can still be beneficial to have all our inputs (and consequently, our batches) to be of uniform size to help with optimizations.<br><br>
In this case, we'll make all our reviews 200 tokens in length (in practice, you can choose a number based on some analysis). So the reviews longer than 200 tokens will be truncated, while the reviews shorter than 200 will be padded with zeroes.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

In [ ]:
MAX_REVIEW_LEN = 200
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=MAX_REVIEW_LEN)

In [ ]:
 # truncated example
 print(X_train[0])

In [ ]:
# padded example
print(X_train[1])

Our training set is prepared. We can now also vectorize and pad our validation set.

In [ ]:
X_val = tokenizer.texts_to_sequences(val_reviews)
X_val = keras.preprocessing.sequence.pad_sequences(X_val, maxlen=MAX_REVIEW_LEN)

Now we need to incorporate the Google News vectors (currently loaded into gensim) into our Keras model. What we'll do is create an embedding matrix that maps each tokenizer integer to its respective word vector.<br><br>
For example, here's the index for the word "good" from the Keras tokenizer and the word vector for "good" from gensim. We want a matrix which maps the index to the vector.


In [ ]:
print(tokenizer.word_index['good'])

In [ ]:
# Part of the vector for the word 'good'.
print(word_vectors['good'][:50])

We'll create this embedding matrix by first initializing a matrix of zeros, then looping over every word in the tokenizer vocabulary and:
1. Checking if the word has a corresponding vector in gensim.
2. If it does, then copy the vector into the matrix row corresponding to the word's index.

In [ ]:
# + 1 to account for padding token.
num_tokens = len(tokenizer.word_index) + 1

# Initialize a matrix of zeroes of size: vocabulary x embedding dimension.
embedding_dim = 300
embedding_matrix = np.zeros((num_tokens, embedding_dim))

for word, i in tokenizer.word_index.items():
  if word_vectors.has_index_for(word):
    embedding_matrix[i] = word_vectors[word].copy()


In [ ]:
# Quick visual check.
print(embedding_matrix[tokenizer.word_index['good']][:50])

We're ready to build our first model using pretrained word vectors. The first layer we'll add is a Keras **embedding** layer which is essentially a trainable lookup table/matrix.<br>
https://keras.io/api/layers/base_layer/#layer-class<br>
https://keras.io/api/layers/core_layers/embedding/<br><br>
In this case, we'll populate the **embedding** layer with the embedding matrix we created, and set *trainable* to True. This means we'll allow the learning algorithm training the classification model to adjust/fine-tune the word vectors as needed for greater performance. This corresponds to one of the scenarios we covered in the slides.

In [ ]:
embedding_layer = layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    input_length=MAX_REVIEW_LEN,
    trainable=True
)

We'll use a simple architecture for this model. Each training example is a sequence of *integers* which gets converted to a sequence of *vectors* (embeddings), but subsequent layers are expecting one vector per review. So we're inserting a **GlobalAveragePooling1D** layer after the embedding layer to average out all the word vectors into a single vector, before sending it further into the network. For classification, this can be pretty effective as a base model approach.<br>
https://keras.io/api/layers/pooling_layers/global_average_pooling1d/<br>

There was no science behind choosing 128 units in the first hidden layer and 64 units in the second hidden layer. The intuition was that the signal would be distilled from 300 dimensions down to 128 dimensions, then down to 64 dimensions before going to output.



In [ ]:
tf.random.set_seed(0)

model = keras.Sequential()

# This layer will output a sequence of 300-dimension *vectors*, one for each element in the input sequence.
model.add(embedding_layer)

# This layer will calculate an average of those vectors.
model.add(layers.GlobalAveragePooling1D())

model.add(layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.random_normal(seed=1)))
model.add(layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.random_normal(seed=1)))
model.add(layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.random_normal(seed=1)))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Here's an example of what's going to happen under the hood to turn review text into a single vector for the dense layers:

In [ ]:
review = "fantastic papaya steak"
print(f"Review: {review}")

review_sequence = tokenizer.texts_to_sequences([review])
print(f"Review as sequence of integers: {review_sequence}")

review_embeddings = embedding_layer(np.array(review_sequence))
print(f"Review embeddings shape: (Batch size: {review_embeddings.shape[0]}, \
Sequence length: {review_embeddings.shape[1]}, \
Embedding size: {review_embeddings.shape[2]})")

# How our document will be presented to the rest of the neural network.
print(f"Average of embeddings (shape): {np.mean(review_embeddings, axis=1).shape}")

When we call the model's *summary* method, note how there are no params for the **GlobalAveragePooling1D** layer.

In [ ]:
model.summary()

We won't use **early stopping** for this run. This way, we'll be able to compare metrics between the train and validation sets.

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=512, validation_data=(X_val, y_val))

In [ ]:
def plot_train_vs_val_performance(history):
  training_losses = history.history['loss']
  validation_losses = history.history['val_loss']

  training_accuracy = history.history['accuracy']
  validation_accuracy = history.history['val_accuracy']

  epochs = range(1, len(training_losses) + 1)

  import matplotlib.pyplot as plt
  fig, (ax1, ax2) = plt.subplots(2)
  fig.set_figheight(15)
  fig.set_figwidth(15)
  fig.tight_layout(pad=5.0)

  # Plot training vs. validation loss.
  ax1.plot(epochs, training_losses, 'bo', label='Training Loss')
  ax1.plot(epochs, validation_losses, 'b', label='Validation Loss')
  ax1.title.set_text('Training vs. Validation Loss')
  ax1.set_xlabel('Epoch')
  ax1.set_ylabel('Loss')
  ax1.legend()

  # PLot training vs. validation accuracy.
  ax2.plot(epochs, training_accuracy, 'bo', label='Training Accuracy')
  ax2.plot(epochs, validation_accuracy, 'b', label='Validation Accuracy')
  ax2.title.set_text('Training vs. Validation Accuracy')
  ax2.set_xlabel('Epoch')
  ax2.set_ylabel('Accuracy')
  ax2.legend()

  plt.show()

In [ ]:
plot_train_vs_val_performance(history)

Let's train the model again using above information.
We'll initialize a new embedding layer and model and train for epochs equalling the point where we saw the validation loss diverge from the training loss.<br>

**NOTE**: We need to initialize a new embedding layer here because we set the *learnable* parameter to **True** in the previous embedding layer. This means the previous embeddings were almost certainly updated by the learning algorithm. So we're re-training a new model now with the original embeddings.

In [ ]:
embedding_layer = layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    input_length=MAX_REVIEW_LEN,
    trainable=True
)

model = keras.Sequential()
model.add(embedding_layer)
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3, batch_size=512, validation_data=(X_val, y_val))

Now that we have a trained model, let's try it on the test data. As we did with the training data, we'll:
1. Replace the labels with 0 for negative sentiment, and 1 for positive.
2. Convert the reviews into a sequence of integers and pad/truncate each review to a fixed length.

In [ ]:
yelp_test = pd.read_csv('yelp_review_polarity_csv/test.csv', names=['sentiment', 'review'])

In [ ]:
yelp_test['sentiment'].replace(to_replace=1, value=0, inplace=True)
yelp_test['sentiment'].replace(to_replace=2, value=1, inplace=True)
yelp_test.head()

In [ ]:
y_test = np.array(yelp_test['sentiment'])
print(y_test)

In [ ]:
X_test = tokenizer.texts_to_sequences(yelp_test['review'])

In [ ]:
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=MAX_REVIEW_LEN)

In [ ]:
model.evaluate(X_test, y_test)

Not bad for a conceptually simple model where we average out a review's word vectors, run it through a few plain hidden layers, and out through a sigmoid function with no regularization and just using defaults for model components (e.g. optimizer settings).<br><br>
We can now use the model for predictions.

In [ ]:
def sentiment(reviews):
  seqs = tokenizer.texts_to_sequences(reviews)
  seqs = keras.preprocessing.sequence.pad_sequences(seqs, maxlen=MAX_REVIEW_LEN)
  return model.predict(seqs)


In [ ]:
# Real reviews from Google Reviews.
pos_review = "The best seafood joint in East Village San Diego!  Great lobster roll, great fish, great oysters, great bread, great cocktails, and such amazing service.  The atmosphere is top notch and the location is so much fun being located just a block away from Petco Park (San Diego Padres Stadium)."
neg_review = "A thoroughly disappointing experience. When you book a Marriott you expect a certain standard. Albany falls way short. Room cleaning has to be booked 24 hours in advance but nobody thought to mention this at check in. The hotel is tired and needs a face-lift. The only bright light in a sea of mediocrity were the pancakes at breakfast. Sadly they weren't enough to save the experience. If you travel to Albany, then do yourself a big favour and book the Westin."

In [ ]:
print(sentiment([pos_review, neg_review]))

We get right predictions for the reviews.